In [1]:
%%datalabframework getfilename

In [2]:
import datalabframework as dlf
logger = dlf.log.initLogger(__name__, kafka_topic="datalab", kafka_servers="kafka:9092")

2017-06-13 19:28:07,853 - jovyan - __main__ - INFO - init - {'project': {'rootpath': '/home/jovyan/work/notebooks', 'main': 'main.ipynb'}, 'notebook': {'filename': 'regression.ipynb', 'filepath': '/home/jovyan/work/notebooks/validate'}, 'datalab': {'framework': '0.1'}}


In [3]:
# EXPORT

from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
# EXPORT

def auc(df, model):
    prediction = model.transform(df)
    evaluator = BinaryClassificationEvaluator()

    metric = evaluator.evaluate(prediction)
    metric_name = evaluator.getMetricName()

    return metric_name, metric

In [5]:
DATA_ROOT = '/home/jovyan/work/data'
d = {   
    'input_sample' : 1.0,
    'input_source' : DATA_ROOT + '/titanic/data/set/train/features.parquet',
    'input_model' : DATA_ROOT + '/titanic/models/lr.model'
}

p = dlf.params.config_fromdict(d)

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(dlf.project.filename()).getOrCreate()
spark.version

'2.1.0'

In [7]:
df = spark.read.parquet(p.input_source)
df.printSchema()
df.show(10, truncate=False)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)



+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|PassengerId|Survived|Pclass|Sex   |Age               |SibSp|Parch|Fare   |Embarked|Title |
+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|1          |0       |3     |male  |22.0              |1    |0    |7.25   |S       |Mr    |
|2          |1       |1     |female|38.0              |1    |0    |71.2833|C       |Mrs   |
|3          |1       |3     |female|26.0              |0    |0    |7.925  |S       |Miss  |
|4          |1       |1     |female|35.0              |1    |0    |53.1   |S       |Mrs   |
|5          |0       |3     |male  |35.0              |0    |0    |8.05   |S       |Mr    |
|6          |0       |3     |male  |28.467886947074096|0    |0    |8.4583 |Q       |Mr    |
|7          |0       |1     |male  |54.0              |0    |0    |51.8625|S       |Mr    |
|8          |0       |3     |male  |2.0               |3    |1    |21.075 |S    

In [8]:
from features import features
df_features = features.feature_vector(df, 
                  'PassengerId', 
                  'Survived', 
                  ['Pclass', 
                   'Sex', 
                   'Age', 
                   'SibSp', 
                   'Parch',
                   'Fare', 
                   'Embarked', 
                   'Title'])

importing Jupyter notebook from /home/jovyan/work/notebooks/features/features.ipynb


In [9]:
from pyspark.ml.classification import LogisticRegressionModel

model = LogisticRegressionModel.load(p.input_model)

Py4JJavaError: An error occurred while calling o50.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jovyan/work/data/titanic/models/lr.model/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:202)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:252)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:250)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:250)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:252)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:250)
	at s

In [ ]:
metric_name, metric = auc(df_features, model)


In [ ]:
print("Evaluation {} : {}".format(metric_name, metric))